In [1]:
import nltk
import pandas as pd
import random
import numpy as np
import string
import math

from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from numpy import array
from collections import Counter
from scipy.sparse import csr_matrix

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
from gensim.corpora import Dictionary

import re

In [2]:
# Use spacy lib
# On https://spacy.io/

import spacy
nlp = spacy.load('fr')

In [3]:
##############
# Parameters #
##############

min_gram = 1
max_gram = 3

# To create ours partitions, we must first know the years which will be the limits
limit_years = [2007, 2010, 2014]

# Ignore words that appear at a frequency less than tresh_freq in the corpus
tresh_freq = 0.8


In [4]:
# Datas preprocessing methods.

# Lemmatisation without poncutations

stemmer = nltk.stem.snowball.FrenchStemmer()
fstw = stopwords.words('french')

# French Stop Words, extraits depuis le fichier stopwords-fr.txt + stopwords french de nltk
sourceFST = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()]+fstw

# Based on ration of french and english stopwords
def isEnglish(article):
    total_fsw = len([x for x in article.split() if x in sourceFST])
    total_esw = len([x for x in article.split() if x in stopwords.words('english')])
    ratio = 100
    if total_fsw != 0:
        ratio = total_esw/total_fsw
    return ratio > 1 and total_esw > 3

def lemmatize(article):
    artiregex = re.sub(" [A-z][A-z] ", " ", article) # word of length < 2
    artiregex = artiregex.lower()
    artiregex = re.sub("(é|è|ê)", "e", artiregex)
    output = []
    outPonc = artiregex.translate(artiregex.maketrans("","", string.punctuation))
    outLem = nlp(outPonc)
    for token in outLem:
        if token.lemma_ not in sourceFST and [x for x in token.lemma_ if x not in "0123456789"] != []:
            output.append(token.lemma_)
    res = ' '.join(output)
    return res

In [ ]:
# Data Reading
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

In [ ]:
# Let's process our corpus, and determine a limit to split it in partitions

# usable[] correspond to our corpus processed
# limits[] let us know when to delimit partitions
limits = []
usable = []

prev_year = data['year'][0]
numArti = 0
for i in range(0, len(data['abstract']), 1):
    #if not null, empty, or whatever (so if there is a abstract):
    if not isinstance(data['abstract'][i], float) and not isEnglish(data['abstract'][i]):
        text = data['abstract'][i]
        if not isinstance(data['title'][i], float):
            text += " "+data['title'][i]

        numArti+=1
        usable.append(stemmer.stem(lemmatize(text)))
        year = data['year'][i]
        if year != prev_year:
            prev_year = year
            if year in limit_years:
                limits.append(numArti)
limits.append(numArti)


In [ ]:
# Display pre-processed datas

vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram), max_df=tresh_freq)
tfidf = vectorizer.fit_transform(usable)

print("nombre d'articles =", len(usable))
print("nombre de mots =", len(tfidf.toarray()[0]))
print("limits =", limits)

usable[0]

/info/etu/m2/i140302/venv/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['quelqu'] not in stop_words.
  sorted(inconsistent))


nombre d'articles = 991
nombre de mots = 133091
limits = [223, 468, 694, 991]


'plateforme objectif permettre citoyen danalyserpar euxmemer tweet politique devenement specifiqu francepour cas lelection presidentiell ideo2017 analyser quasitemps reel message candidat fournir principal caracteristiqueslusage lexiqu politique comparaison entrer candidat ideo2017   plateforme citoyen dediee lanalyse tweet evenement polit'

In [ ]:
# Creation of partitions_tfidf[], which give us the TFIDF of each cluster of each partition
# partitions_tfidf[num_partition][num_doc][num_word]
# Beware, num_doc can't be equals to 1091 (max). You have partitions, so every doc aren't in every partitions
# num_word can be found via vectorizer.get_feature_name()
partitions_tfidf = []
beg = 0
for l in limits:
    last = l
    partitions_tfidf.append([list(x) for x in list(tfidf.toarray())[beg:last]])
    beg = l

In [ ]:
len(partitions_tfidf[0][0])

133091

In [ ]:
# From here, the end use LSA :P Then the code below isn't important

In [ ]:
#params
nb_concepts = 30
min_gram = 1
max_gram = 3

# Creation of cleandocs, which is usable[] with ngrams
cleandocs = []
for t in usable:
    doc = []
    for n in range(min_gram, max_gram+1):
        for gram in ngrams(t.split(), n):
            doc.append(" ".join(gram))
    cleandocs.append(doc)

# Creation of tfidf model, a tool to create ours tfidf
corpus = []
dictionary = corpora.Dictionary(cleandocs)
for doc in cleandocs:
    newVec = dictionary.doc2bow(doc)
    corpus.append(newVec)
tfidf = models.TfidfModel(corpus)

# Creation of partitions_lsa[], which give us the LSA of each partition
partitions_lsa = []
beg = 0
for l in limits:
    last = l
    corpus_tfidf = tfidf[corpus]
    lsi = models.LsiModel(corpus_tfidf, num_topics=nb_concepts, id2word=dictionary)
    corpus_lsi = lsi[corpus_tfidf[beg:last]]
    partitions_lsa.append(corpus_lsi)
    beg = l

In [ ]:
num_partition = 0
for lsa in partitions_lsa:
    print("Partition numéro:",num_partition)
    num_partition+=1
    i=0
    for doc in lsa:
        if (i<3):
            print("document number ", i)
            i+=1
            print(doc)


Partition numéro: 0
document number  0
[(0, 0.02982208116887981), (1, -0.001906143223681982), (2, 0.010527762486456876), (3, 0.00512208054114747), (4, -0.0240719681584426), (5, -0.005643712332486874), (6, -0.009955091977091294), (7, -0.009954155383911407), (8, -0.017674406998599592), (9, -0.0024169278588186544), (10, -0.0001852330263841159), (11, -0.05189196475192622), (12, 0.001499112528902667), (13, 0.008017471966816216), (14, -0.034872470177392735), (15, -0.030681999944283774), (16, 0.044520907777872515), (17, 0.006840765395550134), (18, 0.0009695079689956844), (19, -0.04243464814555533), (20, 0.00746589186631983), (21, -0.01899344996867067), (22, 0.024389782182991832), (23, 0.030600463480148228), (24, -0.007809583555913306), (25, -0.018998973180313038), (26, 0.01497127187502871), (27, -0.03010393486014037), (28, 0.016232599468182927), (29, 0.01620951523807301)]
document number  1
[(0, 0.08978169498851843), (1, 0.009278949774664891), (2, -0.07193406882349739), (3, -0.049958924950165

In [ ]:
# Let's create ours partitions
partitions = []

# You must specify a treshold, to know what are the doc you keep, and what are the doc you drop
tresh = 0.03

for corpus_lsi in partitions_lsa:
    # Let's create ours clusters
    clusters = []

    for i in range(0, nb_concepts):
        dic = {}
        num_doc = 0
        for doc in corpus_lsi:
            if abs(doc[i][1]) > tresh:
                dic[num_doc] = doc[i][1]
            num_doc+=1
        clusters.append(dic)
    partitions.append(clusters)
    
# TODO: it would be nice to know how many articles are in no cluster anymore

In [ ]:
# Display clusters 3 of partition 0 
partitions[0][3]

{1: -0.049958924950165406,
 2: 0.08744356153210764,
 3: -0.03321621232189094,
 11: 0.07050055958917299,
 13: 0.03422501551162143,
 15: 0.03230912163574654,
 21: 0.11285664431396818,
 27: -0.10550485491711832,
 29: 0.044595159264449326,
 30: -0.0306190621928106,
 35: -0.056128342241752865,
 36: 0.0957441200392091,
 41: 0.04519098986834995,
 44: -0.03445520091494497,
 45: 0.04356654060428543,
 46: 0.05346084279427925,
 51: 0.059784304490782744,
 55: 0.03644473963900893,
 56: 0.0986950896418553,
 60: -0.03162306028701053,
 66: -0.032069014959174594,
 70: 0.03399078536635222,
 80: 0.036483227620393244,
 82: -0.04452735144575009,
 83: 0.03444596871834685,
 84: -0.03627525441889498,
 89: -0.048165782102142535,
 92: 0.042760666958574084,
 93: 0.08176969284388126,
 95: 0.04521500397173245,
 98: 0.038611903229600925,
 99: 0.21473160161328592,
 100: -0.04437905694905508,
 103: 0.0844815488506724,
 104: 0.07832763891585205,
 106: -0.055187865477333654,
 108: 0.03183604490658634,
 109: 0.074429919

In [ ]:
nb_labels_by_cluster = 5

# Let's labelize our clusters
# For this, we will use the tfidf matrix

vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram))
tfidf = vectorizer.fit_transform(usable)

# We can access the value in the tfidf using:
#tfidf.toarray().item(num_doc, num_word)
# To know the number of the word searched, we will use:
#vectorizer.vocabulary_[word]

# take less than 8h to compute x)
labels = []
for clusters in partitions:
    l = []
    for clus in clusters:
        first_arti = True
        for article in clus:
            link = abs(clus[article])
            if first_arti:
                coef_list = (tfidf.toarray()[article] * link)
                first = False
            else:
                # the more an article have a high coeficient, the more he is implied in the labeling step
                coef_list += (tfidf.toarray()[article] * link)
        # Now we have coef_list filled by every coeficient in the multiple tfidf
        # Let's find the best ones, to finally get the labels
        res = dict(zip(vectorizer.get_feature_names(), coef_list))

        l.append(Counter(res).most_common(nb_labels_by_cluster))
    labels.append(l)

# TODO: on observe beaucoup de labels identiques entre deux clusters
# Je pense que c'est parce que l'on a trop de clusters, mais j'aimerais en être sûr

In [ ]:
# Display labels
# labels is composed by an array for each partition
labels